In [1]:
import numpy as np
import scipy
from scipy.optimize import curve_fit
import plotly
from plotly import graph_objects as go
import math
from spring_math import *

### In this notebook, we try to find omega given duration when the damping ratio is less than 1 as is done in UIKit API

In [2]:
def simplified(omega, u, E):
    return (u / omega - 1) * np.exp(-E * omega)

In [3]:
def right_hand_side(zeta):
    return 0.001 * np.sqrt(1 - zeta ** 2) / zeta

### u = v0 / zeta, E = duration * zeta. we want to solve the equation simplified(omega, u, E) = right_hand_side to find omega

In [4]:
right_hand_side(0.5)

0.0017320508075688772

In [5]:
omegas = np.linspace(1, 10, 1000)
alphas = simplified(omegas, u=-1, E=1)
go.FigureWidget(go.Scatter(x=omegas, y=alphas))

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '5dce259f-fc0d-4e1b-933d-fa20f3c81336',
 …

### Negative initial velocity, the derivative of simplified is always positive, the second derivative is always negative. We can use Newton's method to find the solution.

In [6]:
omegas = np.linspace(0.8, 10, 1000)
alphas = simplified(omegas, u=1, E=1)
go.FigureWidget(go.Scatter(x=omegas, y=alphas))

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'e775ff6f-ea18-4f44-8e53-9db40f659394',
 …

### Positive initial velocity, may have multiple solutions

In [7]:
def minimum_point(u, E):
    a = -E / 2 + np.sqrt(E ** 2 / 4 + E / u)
    return 1 / a

In [8]:
minimum_point(1, 1)

1.6180339887498947

### Then we find a condition where the solution is just at the minimum point

In [9]:
us = np.linspace(0.5, 5, 1000)
minimum_values = [simplified(minimum_point(u, E=1), u, E=1) for u in us]
go.FigureWidget(go.Scatter(x=us, y=minimum_values))

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'a65d692f-eeec-4f4d-8bce-cb5a75288c6c',
 …

In [10]:
def binary_search(f, x1, x2, epsilon):
    while x2 - x1 > epsilon:
        mid = x1 + (x2 - x1) / 2
        
        if f(mid) == 0:
            return mid
        else:
            if f(x1) * f(mid) < 0:
                x2 = mid
            else:
                x1 = mid
    
    return x1

In [11]:
def f(u):
    return simplified(minimum_point(u, E=1), u, E=1) + right_hand_side(0.5)

In [12]:
binary_search(f, 1, 5, 1e-9)

3.8078992506489158

In [13]:
3.8078992506489158 * 0.5

1.9039496253244579

### duration: 2, dampingRatio: 0.5, initialVelocity: 1.9039496253244579 is what we want to find.